In [8]:
import os
import urllib
import pymongo  # pip install pymongo
from bson import json_util 
from pymongo import MongoClient # Comes with pymongo
import pandas as pd
# read,concatenate, split audio
import pydub
from pydub import AudioSegment
from scipy.io import wavfile
# shuffle data in dataframe
from sklearn.utils import shuffle
# transform audio to feature vector
import librosa
# report result
from sklearn.metrics import classification_report,confusion_matrix
# svm
from sklearn import svm
# read, save model
import joblib
# convert video to audio
import moviepy.editor as mp
import subprocess
# open audio file
import IPython.display as ipd

from tqdm import tqdm

# download video from url link

In [ ]:
# thiết lập folder chứa các thao tác 
# khi tải video sữ được tự động lưu vào folder : "video_file"
# os.chdir('/home/anhnvh/Desktop/Kidtopi/emotion_classification/video_file/')

In [2]:
# đối với các folder_name có khoảng trắng ở giữa.
# Khi ta muốn đọc các file trong folder này thì giữ nguyên link folder path 
# Nếu muốn write 1 file vào folder này, thì cta dùng '\ ' ở trước khoảng trắng trong link folder path 
video_path = '.../emotion_data/video_file/'
audio_path = '.../emotion_data/audio_file/'

In [3]:
# connect to db of mongodb: db
username = 'ha'
password = 'ha123'
databaseName = 'haha'
# connect with authentication
client = MongoClient("mongodb:localhost")
db = client[databaseName]
db.authenticate(username, password) 

True

In [ ]:
# # take all
# query = {}
# # Make a query to the specific DB and Collection
# cursor = db['playback_video'].find(query)

# # Expand the cursor and construct the DataFrame
# df =  pd.DataFrame(list(cursor))
# # take infor of teacher 
# df = df[df.user_role == 0]
# df.url.value_counts().to_frame().shape

In [ ]:
# ls_class_video_url = df.url.unique()
# # download video form url link 
# # đối với các folder_name có khoảng trắng ở giữa
# # Nếu muốn write 1 file vào folder này, thì cta dùng '\ ' ở trước khoảng trắng trong link folder path 
# if ' ' in video_path:
#     video_path_writer = video_path.replace(' ', '\ ')
# else:
#     video_path_writer = video_path
# for i, url in enumerate(ls_class_video_url[150:250]):
#     urllib.request.urlretrieve(url, video_path_writer + 'video{0}.mp4'.format(i + 150))

# extract audio from video

In [ ]:
# thiết lập folder chứa các thao tác 
# khi extract audio từ video thì audio sẽ được tự động lưu vào folder : "audio_file"
# os.chdir('.../emotion_classification/audio_file/')                     

In [4]:
video_files = os.listdir(video_path)
video_files

['video182.mp4',
 'video103.mp4',
 'video104.mp4',
 'video106.mp4',
 'video117.mp4',
 'video124.mp4',
 'video125.mp4',
 'video126.mp4',
 'video143.mp4',
 'video145.mp4',
 'video150.mp4',
 'video152.mp4',
 'video154.mp4',
 'video155.mp4',
 'video156.mp4',
 'video157.mp4',
 'video158.mp4',
 'video159.mp4',
 'video160.mp4',
 'video162.mp4',
 'video165.mp4',
 'video166.mp4',
 'video167.mp4',
 'video169.mp4',
 'video178.mp4',
 'video179.mp4',
 'video181.mp4',
 'video183.mp4',
 'video184.mp4',
 'video186.mp4',
 'video188.mp4',
 'video189.mp4',
 'video191.mp4',
 'video193.mp4',
 'video194.mp4',
 'video195.mp4',
 'video196.mp4',
 'video197.mp4',
 'video200.mp4',
 'video201.mp4',
 'video203.mp4',
 'video204.mp4',
 'video205.mp4',
 'video206.mp4',
 'video208.mp4',
 'video209.mp4',
 'video211.mp4',
 'video213.mp4',
 'video214.mp4',
 'video216.mp4',
 'video217.mp4',
 'video218.mp4',
 'video219.mp4']

In [5]:
# create a function, split_audio_from_video
def extract_audio_from_video(video_name):
    # find the index of '.mp4' in the video name
    idx_extension = video_name.find('.mp4')
    # audio name, it's saved in the folder
    audio_name = video_name[:idx_extension] + '.wav'
    # đối với các folder_name có khoảng trắng ở giữa
    # Nếu muốn write 1 file vào folder này, thì cta dùng '\ ' ở trước khoảng trắng trong link folder path 
    if ' ' in audio_path:
        audio_path_write = audio_path.replace(' ', '\\ ')
    else:
        audio_path_write = audio_path
        
    if ' ' in video_path:
        video_path_writer = video_path.replace(' ', '\\ ')
    else:
        video_path_writer = video_path
    # way 1: https://zulko.github.io/moviepy/ref/VideoClip/VideoClip.html
    # clip = mp.VideoFileClip(video_path + video_name)
    # clip.audio.write_audiofile(audio_path + audio_name
    # way 2: extract audio full time from video
    command = "ffmpeg -i {0} -async 1  {1}".format(video_path_writer + video_name, audio_path_write + audio_name)
    #print(command)
    #print('name : ',video_name)
    subprocess.call(command, shell=True)

In [6]:
# extract audio from video
for video in tqdm(video_files):
    extract_audio_from_video(video)

100%|██████████| 53/53 [05:34<00:00,  5.96s/it]


# Load audio file 

In [ ]:
# open audio file 
# '/home/anhnvh/Desktop/Kidtopi/emotion_classsification/audio_file/video0.wav'
# import IPython.display as ipd
# ipd.Audio(audio_path)

# split audio file into parts 

In [10]:
# level min-second
def split_audio_part(startMin, startSec, endMin, endSec, audio_name, output_format = 'wav'):
    print(audio_name)
    # Time to miliseconds
    startTime = startMin*60*1000+startSec*1000
    endTime = endMin*60*1000+endSec*1000
    if ' ' in audio_path:
        audio_path_write = audio_path.replace(' ', '\\ ')
    else:
        audio_path_write = audio_path

    # Opening file and extracting segment
    song = pydub.AudioSegment.from_wav(audio_path + audio_name)
    extract = song[startTime:endTime]
    # Saving
    # extract.export(audio_part_path + audio_part_name, format= output_format)
    return extract

In [11]:
audio_files = os.listdir(audio_path)
# audio_files

In [9]:
def take_time(startMin, startSec, endMin, endSec, video_name, text):
    d = dict()
    d['startMin'] = float(startMin)
    d['startSec'] = float(startSec)
    d['endMin'] = float(endMin)
    d['endSec'] = float(endSec)
    # Time to miliseconds
    d['startTime'] = float(startMin)*60*1000 + float(startSec)*1000
    d['endTime'] = float(endMin)*60*1000 + float(endSec)*1000
    d['video'] = video_name
    d['text'] = text
    return d

# detect location of praise speed in audio

In [16]:
def convertMillis_to_Time(millis):
    seconds=int((millis/1000)%60)
    minutes=int((millis/(1000*60))%60)
    hours=int((millis/(1000*60*60))%24)
    return '{0}:{1}:{2}'.format(hours, minutes, seconds)

In [9]:
# dataframe contain the loaction in audio appear good, very good, good job, perfect, great, ...
#df_part_time = pd.DataFrame(columns= ['startMin', 'startSec', 'endMin', 'endSec', 'startTime', 'endTime'])
path = '.../emotion_data/'
df_part_time = pd.ExcelFile(path + 'praise_speed_location.xlsx').parse('Sheet1')

In [11]:
df_part_time[df_part_time.video == 'video0.wav']

,startMin,startSec,endMin,endSec,startTime,endTime,video,text,part_name
0,0,44.0,0,46.0,44000,46000,video0.wav,very good,praise_audio_00.wav
1,1,2.0,1,4.0,62000,64000,video0.wav,very good,praise_audio_01.wav
2,1,30.0,1,32.0,90000,92000,video0.wav,good,praise_audio_02.wav
3,1,49.0,1,51.0,109000,111000,video0.wav,very good,praise_audio_03.wav
4,2,21.0,2,23.0,141000,143000,video0.wav,very good,praise_audio_04.wav
5,2,48.0,2,50.0,168000,170000,video0.wav,very good,praise_audio_05.wav
6,5,6.0,5,8.0,306000,308000,video0.wav,good job,praise_audio_06.wav
7,6,47.0,6,49.0,407000,409000,video0.wav,very good,praise_audio_07.wav
8,8,35.0,8,37.0,515000,517000,video0.wav,very good,praise_audio_08.wav
9,10,0.0,10,2.0,600000,602000,video0.wav,very good,praise_audio_09.wav


In [10]:
df_part_time[df_part_time.part_name == 'praise_audio_11.wav']

,startMin,startSec,endMin,endSec,startTime,endTime,video,text,part_name
11,12,3.0,12,5.0,723000,725000,video0.wav,very good,praise_audio_11.wav


In [549]:
# name_video = 'video72.wav'
# ls_time = [27, 16, 27, 18]
# split_audio_part(ls_time[0], ls_time[1], ls_time[2], ls_time[3], name_video, 'test.wav')
# #load audio 
# ipd.Audio('/home/anhnvh/Desktop/Kidtopi/emotion_classification/test_part/test.wav')

audio_name = 'video152.wav'
sample_rate, audio = wavfile.read(audio_path + audio_name)
print("total time in audio (second): ", float("{:10.2f}".format(len(audio) / sample_rate)))
print("total time in audio (Time): ", convertMillis_to_Time(float("{:10.2f}".format(len(audio) / sample_rate))*1000))
ls_time = [9, 46, 9, 47.7]
split_audio_part(ls_time[0], ls_time[1], ls_time[2], ls_time[3], audio_name)

total time in audio (second):  1906.72
total time in audio (Time):  0:31:46
video152.wav


In [550]:
text = 'very good'

In [551]:
df_part_time = df_part_time.append(take_time(ls_time[0], ls_time[1], ls_time[2], ls_time[3], audio_name, text), ignore_index = True)

In [552]:
df_part_time

,startMin,startSec,endMin,endSec,startTime,endTime,video,text,part_name
0,0.0,44.0,0.0,46.0,44000.0,46000.0,video0.wav,very good,praise_audio_00.wav
1,1.0,2.0,1.0,4.0,62000.0,64000.0,video0.wav,very good,praise_audio_01.wav
2,1.0,30.0,1.0,32.0,90000.0,92000.0,video0.wav,good,praise_audio_02.wav
3,1.0,49.0,1.0,51.0,109000.0,111000.0,video0.wav,very good,praise_audio_03.wav
4,2.0,21.0,2.0,23.0,141000.0,143000.0,video0.wav,very good,praise_audio_04.wav
5,2.0,48.0,2.0,50.0,168000.0,170000.0,video0.wav,very good,praise_audio_05.wav
6,5.0,6.0,5.0,8.0,306000.0,308000.0,video0.wav,good job,praise_audio_06.wav
7,6.0,47.0,6.0,49.0,407000.0,409000.0,video0.wav,very good,praise_audio_07.wav
8,8.0,35.0,8.0,37.0,515000.0,517000.0,video0.wav,very good,praise_audio_08.wav
9,10.0,0.0,10.0,2.0,600000.0,602000.0,video0.wav,very good,praise_audio_09.wav


In [344]:
df_part_time = df_part_time.drop(571, axis= 0)
# df_part_time.loc[489, 'text'] = 'good'

In [448]:
def check_part(idx):
    startMin = df_part_time.loc[idx, 'startMin']
    startSec = df_part_time.loc[idx, 'startSec']
    endMin = df_part_time.loc[idx, 'endMin']
    endSec = df_part_time.loc[idx, 'endSec']
    audio_name = df_part_time.loc[idx, 'video']
    print('{0}: {1}:{2}-{3}:{4}'.format(audio_name, startMin, startSec, endMin, endSec))
    return split_audio_part(startMin, startSec, endMin, endSec, audio_name)
check_part(69)

video9.wav: 15.0:10.0-15.0:12.0
video9.wav


In [553]:
df_part_time = df_part_time.reset_index()
df_part_time = df_part_time.drop('index', axis= 1)
df_part_time['part_name'] = ''
for i in df_part_time.index:
    if i <10:
        df_part_time.loc[i, 'part_name'] = 'praise_audio_0{}.wav'.format(str(i))
    else:
        df_part_time.loc[i, 'part_name'] = 'praise_audio_{}.wav'.format(str(i))

In [554]:
writer = pd.ExcelWriter(path + 'praise_speed_location.xlsx', engine= 'xlsxwriter')
df_part_time.to_excel(writer, 'Sheet1', index= False)
writer.save()

In [555]:
df_part_time.text.value_counts()

very good        213
good             134
good job         101
perfect           46
very good job     25
great             20
good work         17
excellent         15
nice               9
well done          6
oh right           5
beautiful          5
Name: text, dtype: int64

# take location of not praise in teacher video

In [12]:

#df_not_praise = pd.DataFrame(columns= ['startMin', 'startSec', 'endMin', 'endSec', 'startTime', 'endTime', 'video'])
df_not_praise = pd.ExcelFile(path + 'not_praise_speed_location.xlsx').parse('Sheet1')

In [ ]:
# df_not_praise = df_not_praise[['startMin', 'startSec', 'endMin', 'endSec', 'startTime', 'endTime', 'video', 'part_name', 'text']]

In [ ]:
# split location is not praise (2s) in the a minute
def split_time_not_praise( startSec, endSec, Min, video_name):
    df = pd.DataFrame(columns= ['startMin', 'startSec', 'endMin', 'endSec', 'startTime', 'endTime', 'video', 'part_name', 'text'])
    text = '0'
    for i in range(startSec, endSec):
        if i<59:
            df = df.append(take_time(Min, i, Min, i+2, video_name, text), ignore_index = True)
        elif i == 59:
            df = df.append(take_time(Min, i, Min + 1 , 1, video_name, text), ignore_index = True)
        else:
            pass
    return df[['startMin', 'startSec', 'endMin', 'endSec', 'startTime', 'endTime', 'video', 'part_name', 'text']]       

In [ ]:
# test part 
audio_name1 = 'video124.wav'
# startmin, startsec, endmin, endsec
ls_time2 = [15, 9, 15, 30]
split_audio_part(ls_time2[0], ls_time2[1], ls_time2[2], ls_time2[3], audio_name1)

In [ ]:
# plit the small duration
text = '0'
df_not_praise = df_not_praise.append(take_time(ls_time2[0], ls_time2[1], ls_time2[2], ls_time2[3], audio_name1, text), ignore_index = True)

In [ ]:
# plit the big duration, devide the mall durations
#  startSec, endSec, Min, video_name
df_not_praise = df_not_praise.append(split_time_not_praise(9, 29, 15, audio_name1), ignore_index= True)

In [ ]:
df_not_praise = df_not_praise.reset_index()
df_not_praise = df_not_praise.drop('index', axis = 1)
df_not_praise

In [ ]:
# df_not_praise = df_not_praise.drop(1483, axis= 0)

In [ ]:
def check_part(idx):
    startMin = df_not_praise.loc[idx, 'startMin']
    startSec = df_not_praise.loc[idx, 'startSec']
    endMin = df_not_praise.loc[idx, 'endMin']
    endSec = df_not_praise.loc[idx, 'endSec']
    audio_name = df_not_praise.loc[idx, 'video']
    print('{0}: {1}:{2}-{3}:{4}'.format(audio_name, startMin, startSec, endMin, endSec))
    return split_audio_part(startMin, startSec, endMin, endSec, audio_name)
check_part(1828)

In [ ]:
# df_not_praise = df_not_praise.reset_index()
# df_not_praise = df_not_praise.drop('index', axis= 1)

In [ ]:
# df_not_praise = df_not_praise.reset_index()
# df_not_praise = df_not_praise.drop('index', axis = 1)
df_not_praise['part_name'] = ''
for i in df_not_praise.index:
    if i<10:
        df_not_praise.loc[i, 'part_name'] = 'not_praise_audio_0{}.wav'.format(str(i))
    else:
        df_not_praise.loc[i, 'part_name'] = 'not_praise_audio_{}.wav'.format(str(i))

In [ ]:
writer = pd.ExcelWriter(path + 'not_praise_speed_location.xlsx', engine= 'xlsxwriter')
df_not_praise.to_excel(writer, 'Sheet1', index= False)
writer.save()

In [ ]:
# remove video
# os.remove(video_path + video_name)